In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORT MODULES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

# LOADING THE DATASET

In [ ]:
df = pd.read_csv('../input/bigmart-sales-data/Train.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# unique values in dataset
df.apply(lambda x: len(x.unique()))

# PREPROCESSING THE DATASET

In [ ]:
df.isnull().sum()

In [ ]:
# categorical attributes
cat_col = []
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)

In [ ]:
cat_col

In [ ]:
cat_col.remove('Item_Identifier')

In [ ]:
cat_col.remove('Outlet_Identifier')

In [ ]:
cat_col

In [ ]:
for col in cat_col:
    print(col)
    print(df[col].value_counts())
    print()

In [ ]:
item_weight_mean = df.pivot_table(values = "Item_Weight", index = 'Item_Identifier')

In [ ]:
item_weight_mean

In [ ]:
miss_bool = df['Item_Weight'].isnull()

In [ ]:
miss_bool

In [ ]:
for i, item in enumerate(df['Item_Identifier']):
    if miss_bool[i]:
        if item in item_weight_mean:
            df['Item_Weight'][i] = item_weight_mean.loc[item]['Item_Weight']
        else:
            df['Item_Weight'][i] = np.mean(df['Item_Weight'])

In [ ]:
df['Item_Weight'].isnull().sum()

In [ ]:
outlet_size_mode = df.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=(lambda x: x.mode()[0]))

In [ ]:
outlet_size_mode

In [ ]:
miss_bool = df['Outlet_Size'].isnull()

In [ ]:
df.loc[miss_bool, 'Outlet_Size'] = df.loc[miss_bool, 'Outlet_Type'].apply(lambda x: outlet_size_mode[x])

In [ ]:
df['Outlet_Size'].isnull().sum()

In [ ]:
sum(df['Item_Visibility'] == 0)

In [ ]:
df.loc[:,'Item_Visibility'].replace([0] , [df['Item_Visibility'].mean()] , inplace = True )

In [ ]:
sum(df['Item_Visibility']==0)

In [ ]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
df['Item_Fat_Content'].value_counts()

# CREATION OF NEW ATTRIBUTES

In [ ]:
df['New_Item_Type'] = df['Item_Identifier'].apply(lambda x: x[:2])

In [ ]:
df['New_Item_Type']

In [ ]:
df['New_Item_Type'] = df['New_Item_Type'].map({'FD':'Food', 'NC':'Non-Consumable', 'DR':'Drinks'})
df['New_Item_Type'].value_counts()

In [ ]:
df.loc[ df ['New_Item_Type'] =='Non-Consumable', 'Item_Fat_Content'] = 'Non-Edible'
df['Item_Fat_Content'].value_counts()

In [ ]:
df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']

In [ ]:
df['Outlet_Years']

In [ ]:
df.head()

# EXPLORATORY DATA ANALYSIS

In [ ]:
sns.distplot(df['Item_Weight'] , color = 'red')

In [ ]:
sns.distplot(df['Item_Visibility'] , color = 'green')

In [ ]:
sns.distplot(df['Item_MRP'] , color = 'purple')

In [ ]:
sns.distplot(df['Item_Outlet_Sales'] , color = 'orange')

In [ ]:
# log transformation:
df['Item_Outlet_Sales'] = np.log(1+df['Item_Outlet_Sales'])

In [ ]:
sns.distplot(df['Item_Outlet_Sales'] , color = 'orange')

In [ ]:
sns.countplot(df["Item_Fat_Content"] , palette="Set3")

In [ ]:
l = list(df['Item_Type'].unique())
chart = sns.countplot(df["Item_Type"],palette="Set3")
chart.set_xticklabels(labels=l, rotation=90)

In [ ]:
sns.countplot(df['Outlet_Establishment_Year'] , palette="Set3")

In [ ]:
sns.countplot(df['Outlet_Size'], palette="Set3")

In [ ]:
sns.countplot(df['Outlet_Location_Type'] , palette="Set3")

In [ ]:
sns.countplot(df['Outlet_Type'] , palette = 'Set3')

# COORELATION MATRIX

In [ ]:
corr = df.corr()

In [ ]:
plt.figure(figsize=(15,5))
sns.heatmap(corr, annot=True )

# LABEL ENCODING

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df['Outlet'] = le.fit_transform(df['Outlet_Identifier'])

In [ ]:
cat_col = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'New_Item_Type']
for col in cat_col:
    df[col] = le.fit_transform(df[col])

# ONEHOT ENCODING

In [ ]:
df = pd.get_dummies(df, columns=['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'New_Item_Type'])

In [ ]:
df.head()

# INPUT SPLIT

In [ ]:
X = df.drop(columns=['Outlet_Establishment_Year', 'Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'])

In [ ]:
y = df['Item_Outlet_Sales']

# MODEL TRAINING

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
def train(model, X, y):
    model.fit(X, y)
    pred = model.predict(X)
    
    cv_score = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
    cv_score = np.abs(np.mean(cv_score))
    
    print("Model Report")
    print("MSE:",mean_squared_error(y,pred))
    print("CV Score:", cv_score)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
model = LinearRegression(normalize=True)
train(model, X, y)
coef = pd.Series(model.coef_, X.columns).sort_values()
coef.plot(kind='bar', title="Model Coefficients",color = 'lightblue')

In [ ]:
model = Ridge(normalize=True)
train(model, X, y)
coef = pd.Series(model.coef_, X.columns).sort_values()
coef.plot(kind='bar', title="Model Coefficients" , color = 'red')

In [ ]:

model = Lasso()
train(model, X, y)
coef = pd.Series(model.coef_, X.columns).sort_values()
coef.plot(kind='bar', title="Model Coefficients" , color ='lightblue')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor()
train(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)
coef.plot(kind='bar', title="Feature Importance" , color ='red')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor()
train(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)
coef.plot(kind='bar', title="Feature Importance" , color = 'lightblue')

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
model = ExtraTreesRegressor()
train(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)
coef.plot(kind='bar', title="Feature Importance" , color = 'red')